In [1]:
import os
import json
import re
from tqdm import tqdm
import pickle
import itertools
import networkx as nx
from matplotlib import pyplot as plt
import networkx

In [2]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

In [3]:
path = '../vc_data/posts/'
files = sorted(os.listdir(path))

In [4]:
N = len(files)

In [5]:
N

199187

In [6]:
with open('companies_vc.txt', 'r') as f:
    companies_vc = f.read()

In [7]:
len(companies_vc.split('\n'))

476

In [8]:
companies_vc = companies_vc.split('\n')

In [9]:
with open('companies_habr.txt', 'r') as f:
    companies_habr = f.read()

In [10]:
len(companies_habr.split('\n'))

25244

In [11]:
companies_habr = companies_habr.split('\n')

In [12]:
companies = set(companies_vc).union(set(companies_habr))

In [13]:
data = 0
with open(path + files[100100]) as json_file:
    data = json.load(json_file)

In [14]:
data.keys()

dict_keys(['message', 'result'])

In [15]:
data['result'].keys()

dict_keys(['id', 'url', 'author', 'badges', 'commentsCount', 'commentsSeenCount', 'favoritesCount', 'cover', 'date', 'dateRFC', 'date_favorite', 'last_modification_date', 'hitsCount', 'intro', 'introInFeed', 'isEnabledComments', 'isEnabledLikes', 'isFavorited', 'isRepost', 'likes', 'subsite', 'similar', 'title', 'type', 'entryContent', 'commentatorsAvatars', 'webviewUrl', 'isPinned', 'canEdit', 'etcControls', 'highlight', 'blocks', 'subscribedToTreads'])

In [16]:
data['result']['likes']

{'is_liked': 0, 'count': 68, 'summ': 68, 'is_hidden': False}

In [17]:
data['result']['intro']

'Видеозвонки мессенджер запустил в августе.'

In [18]:
data['result']['blocks']

[{'type': 'text',
  'data': {'text': 'Видеозвонки мессенджер запустил в августе.',
   'text_truncated': '<<<same>>>'},
  'cover': True,
  'hidden': False,
  'anchor': ''},
 {'type': 'text',
  'data': {'text': 'Telegram добавил функцию демонстрации экрана в видеозвонки в бета-версии для macOS, [заметил](https://t.me/klatel/9) Telegram-канал «Кладезь Telegram».',
   'text_truncated': '<<<same>>>'},
  'cover': False,
  'hidden': False,
  'anchor': ''},
 {'type': 'media',
  'data': {'items': [{'title': '',
     'author': '',
     'image': {'type': 'image',
      'data': {'uuid': 'd6123a51-babf-1774-5482-b758b4114c8d',
       'width': 800,
       'height': 552,
       'size': 1320012,
       'type': 'gif',
       'color': '67666e',
       'hash': '',
       'external_service': []}}}],
   'with_background': True,
   'with_border': False},
  'cover': True,
  'hidden': False,
  'anchor': ''},
 {'type': 'text',
  'data': {'text': 'О возможном появлении функции демонстрации экрана в августе 2020

In [19]:
extracted_words = {}
for j in tqdm(range(N)):
    with open(path + files[j]) as json_file:
        data = json.load(json_file)
    num_blocks = len(data['result']['blocks'])
    extracted_words_set = set()
    
    for i in range(num_blocks):
        if data['result']['blocks'][i]['type'] == 'text' or data['result']['blocks'][i]['type'] == 'header':
            extracted_words_set = extracted_words_set.union(set(re.findall(r'«[\w]+»', data['result']['blocks'][i]['data']['text'])))
            #print(re.findall(r'[\s]?[A-Z][A-Za-z]+[\s]|[\s]?[А-Я][А-Яа-я]+[\s]', data['result']['blocks'][i]['data']['text']))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][A-Za-z]*[\.][A-Z][A-Za-z]*|[А-Я][А-Яа-я]*[\.][А-Я][А-Яа-я]*', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z]+[^a-z]|[А-Я]+[^а-я]', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+|[А-Я][а-я]+', data['result']['blocks'][i]['data']['text'])))

    extracted_words[files[j]] = [item.lower() for item in extracted_words_set]
    #        print(data['result']['blocks'][i]['data']['text'].split())
        

100%|██████████████████████████████████| 199187/199187 [09:33<00:00, 347.02it/s]


In [20]:
extracted_words

{'0000000001': ['нерусский', 'фб ', 'и ', 'отсюда', 'соцсети'],
 '0000000002': ['фб', 'маркетологи', 'русский'],
 '0000000003': ['фб', 'маркетологи', 'русский', 'smm '],
 '0000000004': ['фб', 'маркетологи', 'нерусский'],
 '0000000005': ['фб', 'маркетологи', 'русский', 'smm '],
 '0000000006': ['соцсети', 'фб', 'русский'],
 '0000000007': ['фб', 'маркетологи', 'русский', 'smm '],
 '0000000008': ['ачиверы', 'фб', 'криейторы', 'smm ', 'русский'],
 '0000000009': ['фб', 'маркетологи', 'smm ', 'нерусский'],
 '0000000010': ['ачиверы', 'фб', 'русский', 'smm '],
 '0000000011': ['фб', 'русский', 'smm '],
 '0000000012': ['фб', 'русский', 'smm '],
 '0000000013': ['фб', 'русский'],
 '0000000014': ['фб', 'маркетологи', 'русский'],
 '0000000015': ['ачиверы',
  'фб',
  'маркетологи',
  'криейторы',
  'smm ',
  'русский'],
 '0000000016': ['фб', 'маркетологи', 'русский', 'smm '],
 '0000000017': ['фб', 'русский', 'smm '],
 '0000000018': ['фб', 'маркетологи', 'русский', 'гаджеты'],
 '0000000019': ['фб', 'ма

In [21]:
#with open('extracted_words.pkl', 'wb') as f:
#    pickle.dump(extracted_words, f)

In [22]:
'fl' in companies_vc

False

In [23]:
stop_words = set(['set', 
                  'далее', 
                  'развитие', 
                  'алгоритмы', 
                  'мария', 
                  'red', 
                  'main', 
                  'стартап', 
                  'бюро', 
                  'id', 
                  'поиск', 
                  'ясно',
                  'интерес',
                  'we',
                  'fl',
                  'too',
                  'успех',
                  'система'])

In [24]:
companies = set(companies_vc)

In [25]:
companies.add('mail.ru')
companies.add('mail')
companies.add('mail.ru group')
companies.add('мэйл')
#companies.add('facebook')
#companies.add('meta')
companies = companies - stop_words

In [26]:
extracted_companies = {}
for key in extracted_words.keys():
    for word in extracted_words[key]:
        if word in companies:
            if not key in extracted_companies.keys():
                extracted_companies[key] = []
            extracted_companies[key].append(word)

In [27]:
extracted_companies

{'0000000159': ['вконтакте'],
 '0000000308': ['макс'],
 '0000000318': ['макс'],
 '0000000322': ['макс'],
 '0000000368': ['одноклассники'],
 '0000000381': ['одноклассники'],
 '0000000383': ['одноклассники'],
 '0000000390': ['mail'],
 '0000000393': ['одноклассники'],
 '0000000397': ['рбк'],
 '0000000446': ['spotify'],
 '0000000495': ['tj'],
 '0000000496': ['одноклассники'],
 '0000000527': ['одноклассники'],
 '0000000632': ['одноклассники'],
 '0000000708': ['одноклассники'],
 '0000000732': ['пик'],
 '0000000737': ['ростелеком'],
 '0000000745': ['одноклассники'],
 '0000000754': ['одноклассники'],
 '0000000790': ['одноклассники'],
 '0000000820': ['одноклассники'],
 '0000000823': ['мегаплан', 'яндекс'],
 '0000000840': ['одноклассники'],
 '0000000852': ['одноклассники', 'яндекс', 'вконтакте'],
 '0000000857': ['одноклассники'],
 '0000000885': ['одноклассники'],
 '0000001002': ['mail'],
 '0000001099': ['mail'],
 '0000001143': ['одноклассники'],
 '0000001220': ['одноклассники'],
 '0000001261': [

In [240]:
#with open('extracted_companies.pkl', 'wb') as f:
#    pickle.dump(extracted_companies, f)

In [28]:
edges = []
for key in extracted_companies.keys():
    if len(extracted_companies[key]) > 1:
        for element in itertools.product(*[extracted_companies[key], extracted_companies[key]]):
            edges.append(element)

In [29]:
edges

[('мегаплан', 'мегаплан'),
 ('мегаплан', 'яндекс'),
 ('яндекс', 'мегаплан'),
 ('яндекс', 'яндекс'),
 ('одноклассники', 'одноклассники'),
 ('одноклассники', 'яндекс'),
 ('одноклассники', 'вконтакте'),
 ('яндекс', 'одноклассники'),
 ('яндекс', 'яндекс'),
 ('яндекс', 'вконтакте'),
 ('вконтакте', 'одноклассники'),
 ('вконтакте', 'яндекс'),
 ('вконтакте', 'вконтакте'),
 ('mail', 'mail'),
 ('mail', 'авито'),
 ('авито', 'mail'),
 ('авито', 'авито'),
 ('mail', 'mail'),
 ('mail', 'одноклассники'),
 ('одноклассники', 'mail'),
 ('одноклассники', 'одноклассники'),
 ('mail', 'mail'),
 ('mail', 'одноклассники'),
 ('одноклассники', 'mail'),
 ('одноклассники', 'одноклассники'),
 ('одноклассники', 'одноклассники'),
 ('одноклассники', 'mail'),
 ('mail', 'одноклассники'),
 ('mail', 'mail'),
 ('комитет', 'комитет'),
 ('комитет', 'tj'),
 ('tj', 'комитет'),
 ('tj', 'tj'),
 ('mail', 'mail'),
 ('mail', 'яндекс'),
 ('яндекс', 'mail'),
 ('яндекс', 'яндекс'),
 ('яндекс', 'яндекс'),
 ('яндекс', 'mail'),
 ('mail',

In [30]:
edges_ = {}
for item in tqdm(edges):
    if item[0] != item[1]:
        if (not (item[0], item[1]) in edges_.keys()) and (not (item[1], item[0]) in edges_.keys()):
            edges_[item] = 0.0
        else:
            if (item[0], item[1]) in edges_.keys():
                edges_[item] += 1.0
            elif (item[1], item[0]) in edges_.keys():
                edges_[(item[1], item[0])] += 1.0

100%|███████████████████████████████| 108112/108112 [00:00<00:00, 387840.84it/s]


In [31]:
edges_

{('мегаплан', 'яндекс'): 49.0,
 ('одноклассники', 'яндекс'): 607.0,
 ('одноклассники', 'вконтакте'): 305.0,
 ('яндекс', 'вконтакте'): 1063.0,
 ('mail', 'авито'): 123.0,
 ('mail', 'одноклассники'): 581.0,
 ('комитет', 'tj'): 39.0,
 ('mail', 'яндекс'): 3003.0,
 ('timeless', 'coub'): 1.0,
 ('action', 'точка'): 33.0,
 ('яндекс', 'mail.ru'): 703.0,
 ('mail', 'mail.ru'): 2751.0,
 ('тинькофф', 'mail'): 521.0,
 ('тинькофф', 'markswebb'): 39.0,
 ('mail', 'мэйл'): 27.0,
 ('яндекс', 'тинькофф'): 1831.0,
 ('яндекс', 'badoo'): 67.0,
 ('россельхозбанк', 'райффайзенбанк'): 11.0,
 ('тинькофф', 'уралсиб'): 27.0,
 ('intel', 'microsoft'): 673.0,
 ('intel', 'yota'): 9.0,
 ('intel', 'acer'): 41.0,
 ('microsoft', 'yota'): 25.0,
 ('microsoft', 'acer'): 47.0,
 ('yota', 'acer'): 1.0,
 ('mail', 'okko'): 119.0,
 ('райффайзенбанк', 'тинькофф'): 141.0,
 ('pixonic', 'mail'): 33.0,
 ('pixonic', 'mail.ru'): 27.0,
 ('pixonic', 'intel'): 1.0,
 ('mail', 'intel'): 95.0,
 ('mail.ru', 'intel'): 35.0,
 ('pixonic', 'gett'): 

In [32]:
G = nx.Graph()
for key in edges_.keys():
    G.add_weighted_edges_from([(key[0], key[1], edges_[key])])

In [33]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

In [34]:
number_to_adjust_by = 3
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [35]:
from networkx.algorithms import community
communities = community.greedy_modularity_communities(G, weight='weight', resolution=0.8)

In [36]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
colors = (
'#3288bd',
'#66c2a5',
'#abdda4',
'#e6f598',
'#fee08b',
'#fdae61',
'#f46d43',
'#d53e4f',   
'#8c2d04',
)
#Loop through each community in the network
for community_number, community in enumerate(communities[:8]):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = colors[community_number]

In [37]:
networkx.set_node_attributes(G, modularity_class, 'modularity_class')
networkx.set_node_attributes(G, modularity_color, 'modularity_color')

In [45]:
#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'
edge_color = 'gray'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Companies graph'


#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Companies", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
      #  ("Modularity Color", "$color[swatch]:modularity_color"),
]


#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              plot_width=2000, plot_height=2000,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)



#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.3, line_color=edge_color, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
#save(plot, filename=f"{title}.html")

In [273]:
with open('edges_vc.pkl', 'wb') as f:
    pickle.dump(edges, f)

In [274]:
with open('edges_vc_.pkl', 'wb') as f:
    pickle.dump(edges_, f)

In [272]:
len(G.nodes)

116

In [275]:
len(G.edges)

631

In [38]:
#for Eldar save
networkx.set_node_attributes(G, name='size', values=adjusted_node_size)

In [39]:
networkx.set_node_attributes(G, modularity_color, 'color')

In [40]:
networkx.write_gml(G, "vc_posts.gml")